In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import seaborn as sns
import os
from tqdm import tqdm

%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings(action='ignore')

In [2]:
itemsetitem_trn = pd.read_csv(r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject\dataset\itemset_item_training.csv', names = ['itemset', 'item'])
itemsetitem_trn

,itemset,item
0,8571,18506
1,8571,33644
2,8571,32627
3,5725,14855
4,5725,28037
...,...,...
85817,21913,21835
85818,21913,37137
85819,21913,41653
85820,21913,1682


In [3]:
rating = pd.DataFrame({'rating': [1 for i in range(len(itemsetitem_trn))]})
itemsetitem_rating = pd.concat([itemsetitem_trn, rating], axis=1)
itemsetitem_rating

,itemset,item,rating
0,8571,18506,1
1,8571,33644,1
2,8571,32627,1
3,5725,14855,1
4,5725,28037,1
...,...,...,...
85817,21913,21835,1
85818,21913,37137,1
85819,21913,41653,1
85820,21913,1682,1


In [4]:
itemset_item_rating = itemsetitem_rating.pivot_table('rating', index='itemset', columns='item')
itemset_item_rating

item,0,1,2,3,4,5,6,7,8,9,...,42553,42554,42555,42556,42557,42558,42559,42560,42561,42562
itemset,,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27688,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27689,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27690,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
inter_mat = np.zeros( (len(itemsetitem_rating['itemset'].unique()), len(itemsetitem_rating['item'].unique())) )
for item, itemset, rating in zip(itemsetitem_rating['item'], itemsetitem_rating['itemset'], itemsetitem_rating['rating']):
    inter_mat[itemset][item] = rating
inter_mat

MemoryError: Unable to allocate 8.42 GiB for an array with shape (26549, 42563) and data type float64

In [5]:
itemset_item_rating.fillna(0, inplace = True)
itemset_item_rating.head()

item,0,1,2,3,4,5,6,7,8,9,...,42553,42554,42555,42556,42557,42558,42559,42560,42561,42562
itemset,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
len(itemset_item_rating)

22156

In [6]:
chunk_size = 100000  # 배치 크기 설정
melted_data = []
for chunk in itemset_item_rating:
    melted_chunk = pd.melt(chunk)
    melted_data.append(melted_chunk)

melted_data.head()

AttributeError: 'int' object has no attribute 'columns'

In [10]:
len(itemsetitem_trn['item'].unique())

42563

In [14]:
for item in tqdm(range(len(itemsetitem_trn['item'].unique()))):
    for itemset in range(len(itemsetitem_trn['itemset'].unique())):
        negative_sample = pd.Series([item, itemset, 0], index=['item', 'itemset', 'rating'])
        itemsetitem_rating = pd.concat([itemsetitem_rating, negative_sample.to_frame().T], ignore_index=True)

itemsetitem_rating

  0%|          | 54/42563 [2:22:26<1868:52:25, 158.27s/it]


KeyboardInterrupt: 

In [13]:
unique_items = itemsetitem_trn['item'].unique()
unique_itemsets = itemsetitem_trn['itemset'].unique()

negative_samples = pd.DataFrame(
    {'item': unique_items.repeat(len(unique_itemsets)),
     'itemset': list(unique_itemsets) * len(unique_items),
     'rating': 0}
)

itemsetitem_rating = pd.concat([itemsetitem_rating, negative_samples], ignore_index=True)

MemoryError: Unable to allocate 7.03 GiB for an array with shape (943025828,) and data type int64

In [7]:
itemset_item_melt = pd.melt(itemset_item_rating)
itemset_item_melt.head()

MemoryError: Unable to allocate 7.03 GiB for an array with shape (1, 943025828) and data type int64

In [4]:
def preprocessing_df(df):

    num_items = df.item.unique().shape[0]
    num_itemsets = df.itemset.unique().shape[0]
    
    return df, num_items, num_itemsets

In [5]:
data, num_items, num_itemsets = preprocessing_df(itemsetitem_trn)
sparsity = 1 - len(data) / (num_items * num_itemsets)
print(f'number of items: {num_items}, number of itemsets: {num_itemsets}')
print(f'matrix sparsity: {sparsity:f}')
print(data.head())

number of items: 42563, number of itemsets: 22156
matrix sparsity: 0.999909
   itemset   item
0     8571  18506
1     8571  33644
2     8571  32627
3     5725  14855
4     5725  28037


In [44]:
inter_mat = np.zeros((len(itemsetitem_rating['itemset'].unique()), len(itemsetitem_rating['item'].unique())))
for itemset, item, rating in zip(itemsetitem_rating['itemset'], itemsetitem_rating['item'], itemsetitem_rating['rating']):
    inter_mat[itemset][item] = rating

inter_mat

MemoryError: Unable to allocate 8.42 GiB for an array with shape (26549, 42563) and data type float64

In [6]:
class Dataset(Dataset):
    def __init__(self, df, mode='train'):
        self.df = df
        self.mode = mode
        self.train_size = 0.8
        self.val_size = 0.2

        self.num_items = len(self.df['item'].unique())
        self.num_itemset = len(self.df['itemset'].unique())

        n_val = int(self.num_itemset * self.val_size)

        np.random.seed(42)

        val_indices = set(np.random.choice(range(self.num_itemset), n_val, replace = False))
        train_indices = set(range(self.num_itemset)) - val_indices 

        if self.mode == 'train':
            self.df = self.df[self.df['itemset'].isin(train_indices)]
        elif self.mode == 'val':
            self.df = self.df[self.df['itemset'].isin(val_indices)]

        self.item = torch.tensor(self.df['item'].values)
        self.itemset = torch.tensor(self.df['itemset'].values)
        self.ratings = torch.tensor(self.df['rating'].values)
        self.inter_mat = self.make_inter_mat()

    def make_inter_mat(self):
        inter_mat = self.df.pivot_table('rating', index='itemset', columns='item')
        inter_mat.fillna(0, inplace = True)
        inter_mat = inter_mat.values
        return inter_mat
        
    def __len__(self):
        return len(self.inter_mat)
    
    def __getitem__(self, index):
        # inter_mat = torch.tensor(self.inter_mat[self.users[index]]).float()
        inter_mat = torch.tensor(self.inter_mat[index]).float()

        return index, inter_mat

In [7]:
class AutoRec(nn.Module):
    def __init__(self, num_hidden, num_items, dropout=0.2):
        super(AutoRec, self).__init__()
        self.encoder = nn.Linear(num_items, num_hidden)
        self.activate = nn.Sigmoid()
        self.decoder = nn.Linear(num_hidden, num_items)
        self.dropout = nn.Dropout(dropout)

    def forward(self, mat):
        hidden = self.dropout(self.activate(self.encoder(mat)))
        pred = self.activate(self.decoder(hidden))
        
        return pred

In [8]:
def train(model, train_iter, criterion, optm, device):
    model.train() # to train mode
    loss_sum = 0
    loss_cal_count = 0

    for _, inter_mat in train_iter:
        inter_mat = inter_mat.to(device)    

        preds = model(inter_mat)
        loss = criterion(preds, inter_mat)

        # Update
        optm.zero_grad()      # reset gradient 
        loss.backward()      # backpropagate
        optm.step()      # optimizer update
        
        if not np.isnan(loss.cpu().detach().numpy()):
            loss_cal_count += 1
            loss_sum += loss.item()
            
    loss_avg = loss_sum / loss_cal_count

    return loss_avg

In [9]:
def validation(model, valid_iter, criterion, device):
    
    # np.random.seed(42)
    
    with torch.no_grad():
        model.eval()
        loss_sum = 0
        loss_cal_count = 0
        
        for _, inter_mat in valid_iter:
            inter_mat = inter_mat.to(device)
            
            # mask된 input으로 prediction 후 기존 정답 활용 loss값 계산
            preds = model(inter_mat)
            loss = criterion(preds, inter_mat)
            
            if not np.isnan(loss.cpu().numpy()):
                loss_cal_count += 1
                loss_sum += loss.item()
      
    loss_avg = loss_sum / loss_cal_count

    return loss_avg

In [10]:
class AutoRec_loss_fn(nn.Module):

    def __init__(self, loss_fn):
        super(AutoRec_loss_fn, self).__init__()
        self.loss_fn = loss_fn
    
    def forward(self, pred, y):
        y_for_compute = y.clone().to('cpu')
        index = np.where(y_for_compute > 0) # FILL HERE : USE np.where & y_for_compute. WARNING: y를 사용 시, y의 device가 gpu일 경우 오류 발생 #
        loss = self.loss_fn(pred[index], y[index])
        return loss

In [11]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_epochs = 50
lr = 0.001
batch_size = 16

PATH = r'C:\Users\a3179\OneDrive\바탕 화면\datamining_termproject/model/autorec.pt'
patience = 5

num_items = len(itemsetitem_rating['item'].unique())
num_hidden = 40

In [12]:
train_dataset = Dataset(df=itemsetitem_rating, mode='train')
val_dataset = Dataset(df=itemsetitem_rating, mode='val')

train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True) 
val_dataloader = DataLoader(dataset=val_dataset, batch_size=1, shuffle=False)

In [13]:
model = AutoRec(num_hidden, num_items).to(device)
loss = nn.BCELoss().to(device)
optm = optim.Adam(model.parameters(),lr=lr)

In [14]:
class EarlyStopping:
    """주어진 patience 이후로 validation loss가 개선되지 않으면 학습을 조기 중지"""
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt'):
        """
        Args:
            patience (int): validation loss가 개선된 후 기다리는 기간
                            Default: 7
            verbose (bool): True일 경우 각 validation loss의 개선 사항 메세지 출력
                            Default: False
            delta (float): 개선되었다고 인정되는 monitered quantity의 최소 변화
                            Default: 0
            path (str): checkpoint저장 경로
                            Default: 'checkpoint.pt'
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path

    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''validation loss가 감소하면 모델을 저장한다.'''
        if self.verbose:
            print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [15]:
train_losses = []
val_losses = []

early_stopping = EarlyStopping(patience = patience, verbose = True, path=PATH)

for epoch in tqdm(range(num_epochs)):

    train_loss = train(model, train_dataloader, loss, optm, device)
    val_loss = validation(model, val_dataloader, loss, device)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    
    early_stopping(val_loss, model)

    if early_stopping.early_stop:
        print("Early stopping")
        break
        
    print(f'epoch: {epoch}, train Loss: {train_loss:.4f}, test Loss: {val_loss:.4f}')

plt.figure(figsize=(10,5))
plt.plot(train_losses,label="train_loss")
plt.plot(val_losses,label="val_loss")
plt.xlabel("epoch")
plt.ylabel("Loss")
plt.legend()
plt.show()

  0%|          | 0/50 [00:00<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (16x30849 and 42563x40)